**IMPORTANT** 

- For requirements and initial setup go to https://github.com/OliveiraEdu/OpenScience/Readme.md;
- To execute the notebook run all cells.

# Open Science Platform Notebook 1 -User Account Creation

## Activities

1 - Deploys a smart contract into the Iroha 1 blockchain for account creation;

2 - Creates a user account into the blockchain;

3 - Queries the blockchain to confirm the succesful creation of the account;

4 - Deploys a smart contract into the Iroha 1 blockchain for details (attributes) setting;

5 - Sets details for the user account;

6 - Queries the blockchain again to confirm the succesfull details setting.

User accounts and their respective attributes are randomly picked, every account succesfully created is dumped to `datasets/accounts.json` for later use and evidence collection.  

### Sequence Diagram

```mermaid

%%{
  init: {
    'theme': 'base',
    'themeVariables': {
      'primaryColor': '#ffffff',
      'primaryTextColor': '#000000',
      'primaryBorderColor': '#000000',
      'lineColor': '#000000',
      'secondaryColor': '#f4f4f4',
      'secondaryTextColor': '#000000',
      'tertiaryColor': '#d3d3d3',
      'tertiaryTextColor': '#000000',
      'background': '#ffffff',
      'actorBkg': '#B4B4B4',
      'actorTextColor': '#000000',
      'actorBorder': '#000000',
      'actorLineColor': '#000000',
      'signalColor': '#000000',
      'signalTextColor': '#000000',
      'activationBorderColor': '#000000',
      'activationBkgColor': '#d3d3d3',
      'sequenceNumberColor': '#000000',
      'noteBkgColor': '#F0F0F0',
      'noteTextColor': '#000000',
      'noteBorderColor': '#000000'
    }
  }
}%%

sequenceDiagram
    participant Platform as "Platform"
    participant Blockchain as "Iroha 1 Blockchain"

    Note over Platform, Blockchain: Deploy smart contract for account creation
    Platform->>Blockchain: Deploy Smart Contract
    Blockchain-->>Platform: Smart Contract Deployed Successfully
    
    Note over Platform, Blockchain: Account creation
    Platform->>Blockchain: Create Account
    Blockchain-->>Platform: Account Created Successfully

    Note over Platform, Blockchain: Queries the blockchain to confirm account creation
    Platform->>Blockchain: Query Account Details
    Blockchain-->>Platform: Query Response
    
    
    Note over Platform, Blockchain: Deploy smart contract for details setting
    Platform->>Blockchain: Deploy Smart Contract
    Blockchain-->>Platform: Smart Contract Deployed Successfully

    Note over Platform, Blockchain: Set user details
    Platform->>Blockchain: Set User Details in Blockchain
    Blockchain-->>Platform: User Details Set Successfully

    Note over Platform, Blockchain: Queries the blockchain to confirm details setting
    Platform->>Blockchain: Query Account Details
    Blockchain-->>Platform: Query Response
```

In [1]:
from name_generator import left, right
from utilities import *
from dump_to_json import *
from iroha_helper import *



In [2]:
DOMAIN = "test"

1 - Deploys a smart contract into the Iroha 1 blockchain for account creation;

In [3]:
# - Deploy contract
hash = create_account_contract()

2025-01-19 23:53:18.476 | INFO     | integration_helpers:tracer:34 - 	Entering "create_account_contract"
2025-01-19 23:53:18.482 | INFO     | iroha_helper:create_account_contract:41 - ('STATELESS_VALIDATION_SUCCESS', 1, 0)
2025-01-19 23:53:18.482 | INFO     | iroha_helper:create_account_contract:41 - ('ENOUGH_SIGNATURES_COLLECTED', 9, 0)
2025-01-19 23:53:19.266 | INFO     | iroha_helper:create_account_contract:41 - ('STATEFUL_VALIDATION_SUCCESS', 3, 0)
2025-01-19 23:53:19.302 | INFO     | iroha_helper:create_account_contract:41 - ('COMMITTED', 5, 0)
2025-01-19 23:53:19.303 | INFO     | integration_helpers:tracer:36 - 	Leaving "create_account_contract"


2 - Creates a user account into the blockchain.

In [4]:
user_private_key = IrohaCrypto.private_key()
user_public_key = IrohaCrypto.derive_public_key(user_private_key).decode("utf-8")

# - Random generated synthetic user data 
user_account_left = random.choice(left)
user_account_right = random.choice(right)
user_account_short_id = f"{user_account_left}_{user_account_right}"
user_account_full_name = ((f"{user_account_left}").capitalize())+" "+((f"{user_account_right}").capitalize())
user_account_email = f"{user_account_left}_{user_account_right}"+"@email.com"
user_account_institution = print_random_from_second_column("datasets/world-universities.csv")
user_account_orcid = generate_orcid()
user_role = set_random_role()
logger.info(f"User Role: {user_role}")

user_account = UserAccount(
    account_id=f"{user_account_short_id}@{DOMAIN}",
    full_name=user_account_full_name,
    email=user_account_email,
    institution=user_account_institution,
    orcid=user_account_orcid,
    role=user_role,
    public_key=user_public_key,
)
logger.info(f"Creating account with name: {user_account_short_id}")

# - Create account
address = integration_helpers.get_engine_receipts_address(hash)
create_user_account(address, user_account_short_id, DOMAIN, user_public_key, user_account)

# - Get account info
hash = get_account(address, user_account_short_id, DOMAIN)
address = integration_helpers.get_engine_receipts_result(hash)


2025-01-19 23:53:19.350 | INFO     | __main__:<module>:13 - User Role: author
2025-01-19 23:53:19.352 | INFO     | __main__:<module>:24 - Creating account with name: dreamy_johnson
2025-01-19 23:53:19.353 | INFO     | integration_helpers:tracer:34 - 	Entering "get_engine_receipts_address"
2025-01-19 23:53:19.372 | INFO     | integration_helpers:tracer:36 - 	Leaving "get_engine_receipts_address"
2025-01-19 23:53:19.373 | INFO     | integration_helpers:tracer:34 - 	Entering "create_user_account"
2025-01-19 23:53:19.394 | INFO     | iroha_helper:create_user_account:81 - ('STATELESS_VALIDATION_SUCCESS', 1, 0)
2025-01-19 23:53:19.396 | INFO     | iroha_helper:create_user_account:81 - ('ENOUGH_SIGNATURES_COLLECTED', 9, 0)
2025-01-19 23:53:22.331 | INFO     | iroha_helper:create_user_account:81 - ('STATEFUL_VALIDATION_SUCCESS', 3, 0)
2025-01-19 23:53:22.363 | INFO     | iroha_helper:create_user_account:81 - ('COMMITTED', 5, 0)
2025-01-19 23:53:22.367 | INFO     | dump_to_json:dump_to_json_ld:

2025-01-19 10:31:37.981 | INFO     | iroha_helper:create_user_account:81 - ('STATELESS_VALIDATION_SUCCESS', 1, 0)
2025-01-19 10:31:37.983 | INFO     | iroha_helper:create_user_account:81 - ('ENOUGH_SIGNATURES_COLLECTED', 9, 0)
2025-01-19 10:31:40.906 | INFO     | iroha_helper:create_user_account:81 - ('STATEFUL_VALIDATION_SUCCESS', 3, 0)
2025-01-19 10:31:40.933 | INFO     | iroha_helper:create_user_account:81 - ('COMMITTED', 5, 0)
2025-01-19 10:31:40.938 | INFO     | dump_to_json:dump_to_json_ld:82 - Appended new entry to file 'datasets/accounts.json', current total entries: 10
2025-01-19 10:31:40.939 | INFO     | integration_helpers:tracer:36 - 	Leaving "create_user_account"
2025-01-19 10:31:40.940 | INFO     | integration_helpers:tracer:34 - 	Entering "get_account"
2025-01-19 10:31:40.959 | INFO     | iroha_helper:get_account:234 - ('STATELESS_VALIDATION_SUCCESS', 1, 0)
2025-01-19 10:31:40.961 | INFO     | iroha_helper:get_account:234 - ('ENOUGH_SIGNATURES_COLLECTED', 9, 0)
2025-01-1

3 - Queries the blockchain to confirm the succesful creation of the account.

In [5]:
#Query - GetAccountDetail

details = get_account_detail(f"{user_account_short_id}@{DOMAIN}")

logger.info(f'User Account id = {user_account_short_id}@{DOMAIN}, {details}')

2025-01-19 23:53:25.477 | INFO     | __main__:<module>:5 - User Account id = dreamy_johnson@test, {}


4 - Deploys a smart contract into the Iroha 1 blockchain for details (attributes) setting.

In [6]:
# - Deploys the contract
hash = create_detail_contract()
address = integration_helpers.get_engine_receipts_address(hash)
integration_helpers.get_engine_receipts_result(hash)

2025-01-19 23:53:25.487 | INFO     | integration_helpers:tracer:34 - 	Entering "create_detail_contract"
2025-01-19 23:53:25.506 | INFO     | iroha_helper:create_detail_contract:55 - ('STATELESS_VALIDATION_SUCCESS', 1, 0)
2025-01-19 23:53:25.510 | INFO     | iroha_helper:create_detail_contract:55 - ('ENOUGH_SIGNATURES_COLLECTED', 9, 0)
2025-01-19 23:53:28.449 | INFO     | iroha_helper:create_detail_contract:55 - ('STATEFUL_VALIDATION_SUCCESS', 3, 0)
2025-01-19 23:53:28.479 | INFO     | iroha_helper:create_detail_contract:55 - ('COMMITTED', 5, 0)
2025-01-19 23:53:28.481 | INFO     | integration_helpers:tracer:36 - 	Leaving "create_detail_contract"
2025-01-19 23:53:28.482 | INFO     | integration_helpers:tracer:34 - 	Entering "get_engine_receipts_address"
2025-01-19 23:53:28.500 | INFO     | integration_helpers:tracer:36 - 	Leaving "get_engine_receipts_address"
2025-01-19 23:53:28.502 | INFO     | integration_helpers:tracer:34 - 	Entering "get_engine_receipts_result"
2025-01-19 23:53:28.5

5 - Sets details for the user account:

- Full Name
- Email
- Institution
- ORCID
- Public key
- Role

In [7]:
process_account(address, f"{user_account_short_id}@{DOMAIN}")


2025-01-19 23:53:28.532 | INFO     | iroha_helper:process_account:339 - {'@type': 'foaf:Person', 'foaf:name': 'Dreamy Johnson', 'foaf:mbox': 'dreamy_johnson@email.com', 'foaf:organization': {'@type': 'foaf:Organization', 'foaf:name': 'Georg-August Universität Göttingen'}, 'schema:identifier': {'@type': 'PropertyValue', 'propertyID': 'ORCID', 'value': '0552-2296-3197-X'}, 'foaf:holdsAccount': {'schema:identifier': 'dreamy_johnson@test', 'schema:roleName': 'author', 'schema:publicKey': '473a259b26ee9afa73b9be7a43a00ffce6ea1753758603c36f96e4bb6f30e3e9'}}
2025-01-19 23:53:28.565 | INFO     | iroha_helper:process_account:343 - user_json_ld_cid :, QmcVgzyDbnHEPFv6dbRGjgmcip4ZCbd7hfryXv6zYsif5m
2025-01-19 23:53:28.567 | INFO     | iroha_helper:tracer:265 - 	Entering "set_account_detail"
2025-01-19 23:53:28.582 | INFO     | iroha_helper:set_account_detail:317 - None
2025-01-19 23:53:28.584 | INFO     | iroha_helper:set_account_detail:319 - ('ENOUGH_SIGNATURES_COLLECTED', 9, 0)
2025-01-19 23:53

6 - Queries the blockchain again to confirm the succesfull details setting.

In [8]:
#Query - GetAccountDetail

details = get_account_detail(f"{user_account_short_id}@{DOMAIN}")

logger.info(f'User Account id = {user_account_short_id}@{DOMAIN}, {details}')

2025-01-19 23:53:31.559 | INFO     | __main__:<module>:5 - User Account id = dreamy_johnson@test, { "admin@test" : { "user_json_ld_cid" : "QmcVgzyDbnHEPFv6dbRGjgmcip4ZCbd7hfryXv6zYsif5m" } }


7 - Retrieves User Metadata (JSON-LD formatted) from IPFS

In [9]:
# Convert the JSON string to a Python dictionary
details_dict = json.loads(details)

# Now you can access the specific key like this
user_metadata_cid = details_dict["admin@test"]["user_json_ld_cid"]
logger.info(f"User Metadata CID: {user_metadata_cid}")

user_metadata = download_json_from_ipfs(user_metadata_cid)
logger.info(f"User Metadata: {user_metadata}")

2025-01-19 23:53:31.570 | INFO     | __main__:<module>:6 - User Metadata CID: QmcVgzyDbnHEPFv6dbRGjgmcip4ZCbd7hfryXv6zYsif5m
2025-01-19 23:53:31.578 | INFO     | __main__:<module>:9 - User Metadata: {'@type': 'foaf:Person', 'foaf:holdsAccount': {'schema:identifier': 'dreamy_johnson@test', 'schema:publicKey': '473a259b26ee9afa73b9be7a43a00ffce6ea1753758603c36f96e4bb6f30e3e9', 'schema:roleName': 'author'}, 'foaf:mbox': 'dreamy_johnson@email.com', 'foaf:name': 'Dreamy Johnson', 'foaf:organization': {'@type': 'foaf:Organization', 'foaf:name': 'Georg-August Universität Göttingen'}, 'schema:identifier': {'@type': 'PropertyValue', 'propertyID': 'ORCID', 'value': '0552-2296-3197-X'}}
